# Evidence DataSet

In [4]:
import pandas as pd
import numpy as np

In [5]:
from operator import itemgetter

In [6]:
dfs = pd.read_html(r"https://en.wikipedia.org/wiki/2022_FIFA_World_Cup#Teams")
from collections.abc import Iterable

for i in range(len(dfs)):
    df = dfs[i]
    cols = list(df.columns.values)
    
    if isinstance(cols[0], Iterable):
        if any("Tie-breaking criteria" in c for c in cols):
            start_pos = i+1

        if any("Match 46" in c for c in cols):
            end_pos = i+1
matches = []
groups = ["A", "B", "C", "D", "E", "F", "G", "H"]
group_count = 0 

table = {}
#TABLE -> TEAM, POINTS, WIN PROBS (CRITERIO DE DESEMPATE)
table[groups[group_count]] = [[a.split(" ")[0], 0, []] for a in list(dfs[start_pos].iloc[:, 1].values)]

for i in range(start_pos+1, end_pos, 1):
    if len(dfs[i].columns) == 3:
        team_1 = dfs[i].columns.values[0]
        team_2 = dfs[i].columns.values[-1]
        
        matches.append((groups[group_count], team_1, team_2))
    else:
        group_count+=1
        table[groups[group_count]] = [[a, 0, []] for a in list(dfs[i].iloc[:, 1].values)]

In [7]:
WorldCup=pd.DataFrame(columns= ["Group","home_team","away_team",
                                "home_avg_goal","away_avg_goal",
                                "home_rank","away_rank","home_age","away_age",
                                "home_OVA","home_ATT","home_MID","home_DEF",
                                "away_OVA","away_ATT","away_MID","away_DEF",
                                
                                "d_Age","d_rank","d_OVA","d_ATT",
                                "d_MID","d_DEF","d_avg_goal","Class"])

dfmatches=pd.DataFrame(matches, columns= ["Group","home_team","away_team"])
WorldCup[["Group","home_team","away_team"]]=dfmatches


In [8]:
WC_teams=["Ecuador", "Netherlands", "Qatar", "Senegal",
          "England", "Iran", "United States" , "Wales", 
          "Argentina", "Mexico", "Poland" , "Saudi Arabia" , 
          "Australia", "Denmark", "France" , "Tunisia",
          "Costa Rica", "Germany", "Japan" , "Spain",
          "Belgium", "Canada", "Croatia" , "Morocco" ,
          "Brazil", "Cameroon", "Serbia" , "Switzerland",
          "Ghana", "Portugal", "South Korea" , "Uruguay"]

WorldCup.head()


,Group,home_team,away_team,home_avg_goal,away_avg_goal,home_rank,away_rank,home_age,away_age,home_OVA,...,away_MID,away_DEF,d_Age,d_rank,d_OVA,d_ATT,d_MID,d_DEF,d_avg_goal,Class
0,A,Qatar,Ecuador,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,A,Senegal,Netherlands,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A,Qatar,Senegal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,A,Netherlands,Ecuador,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,A,Ecuador,Senegal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Age

In [9]:
Age_full=pd.read_csv("/Users/macbook/Desktop/Uni/Quarter 2/ML/World Cup Project/age_dataframe.csv")

Age_list=[]
for i in range(len(Age_full)):

    if (Age_full.Country.iloc[i] == "USA" ):
        Age_full.Country.iloc[i] = "United States"
        
    if ((Age_full.Country.iloc[i] in WC_teams) and (Age_full.Year.iloc[i]==2022)) :
        Age_list.append(Age_full.iloc[i])
Age=pd.DataFrame(Age_list)
Age.head()

/var/folders/s5/4b25bvq14pqbm1qxsg0hhlr40000gn/T/ipykernel_50087/4139868534.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Age_full.Country.iloc[i] = "United States"


,Year,Country,Average Age
143,2022,Argentina,27.3
191,2022,Australia,27.4
319,2022,Belgium,27.5
447,2022,Brazil,27.2
559,2022,Cameroon,25.7


In [10]:
dfhome = dfmatches.merge(Age, how='inner', left_on='home_team', right_on='Country')
dfinal = dfhome.merge(Age, how='inner', left_on='away_team', right_on='Country')
dfinal.head()

,Group,home_team,away_team,Year_x,Country_x,Average Age_x,Year_y,Country_y,Average Age_y
0,A,Qatar,Ecuador,2022,Qatar,26.7,2022,Ecuador,25.8
1,A,Netherlands,Ecuador,2022,Netherlands,26.9,2022,Ecuador,25.8
2,A,Qatar,Senegal,2022,Qatar,26.7,2022,Senegal,25.1
3,A,Ecuador,Senegal,2022,Ecuador,25.8,2022,Senegal,25.1
4,A,Senegal,Netherlands,2022,Senegal,25.1,2022,Netherlands,26.9


In [11]:
WorldCup["home_age"]=dfinal["Average Age_x"]
WorldCup["away_age"]=dfinal["Average Age_y"]
WorldCup.head()

,Group,home_team,away_team,home_avg_goal,away_avg_goal,home_rank,away_rank,home_age,away_age,home_OVA,...,away_MID,away_DEF,d_Age,d_rank,d_OVA,d_ATT,d_MID,d_DEF,d_avg_goal,Class
0,A,Qatar,Ecuador,NaN,NaN,NaN,NaN,26.7,25.8,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,A,Senegal,Netherlands,NaN,NaN,NaN,NaN,26.9,25.8,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A,Qatar,Senegal,NaN,NaN,NaN,NaN,26.7,25.1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,A,Netherlands,Ecuador,NaN,NaN,NaN,NaN,25.8,25.1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,A,Ecuador,Senegal,NaN,NaN,NaN,NaN,25.1,26.9,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Ranking

In [12]:
Ranking_full=pd.read_csv("/Users/macbook/Desktop/Uni/Quarter 2/ML/World Cup Project/fifa_ranking-2022-10-06.csv")
Ranking_full["Year"]=[int(i[0:4]) for i in Ranking_full.rank_date] 
Ranking_full["Month"]=[int(i[5:7]) for i in Ranking_full.rank_date] 

Ranking_2022=Ranking_full[Ranking_full.Year == 2022]
Ranking_2000=Ranking_2022[Ranking_2022.Month == 10]
Ranking_list=[]
for i in range(len(Ranking_2000)):
    if (Ranking_2000.country_full.iloc[i] == "IR Iran" ):
        Ranking_2000.country_full.iloc[i] = "Iran"
    if (Ranking_2000.country_full.iloc[i] == "USA" ):
        Ranking_2000.country_full.iloc[i] = "United States"
    if (Ranking_2000.country_full.iloc[i] == "Korea Republic" ):
        Ranking_2000.country_full.iloc[i] = "South Korea"
        
    if (Ranking_2000.country_full.iloc[i] in WC_teams):
        Ranking_list.append(Ranking_2000.iloc[i])
Ranking=pd.DataFrame(Ranking_list)
Ranking.head()

/var/folders/s5/4b25bvq14pqbm1qxsg0hhlr40000gn/T/ipykernel_50087/2651263223.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Ranking_2000.country_full.iloc[i] = "South Korea"
/var/folders/s5/4b25bvq14pqbm1qxsg0hhlr40000gn/T/ipykernel_50087/2651263223.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Ranking_2000.country_full.iloc[i] = "United States"
/var/folders/s5/4b25bvq14pqbm1qxsg0hhlr40000gn/T/ipykernel_50087/2651263223.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_gui

,rank,country_full,country_abrv,total_points,previous_points,rank_change,confederation,rank_date,Year,Month
63809,1,Brazil,BRA,1841.30,1837.56,0,CONMEBOL,2022-10-06,2022,10
63811,28,South Korea,KOR,1530.30,1526.02,0,AFC,2022-10-06,2022,10
63813,30,Tunisia,TUN,1507.54,1507.86,0,CAF,2022-10-06,2022,10
63814,31,Costa Rica,CRC,1503.59,1500.06,-3,CONCACAF,2022-10-06,2022,10
63822,38,Australia,AUS,1488.72,1483.73,-1,AFC,2022-10-06,2022,10


In [13]:
rankdfhome = dfmatches.merge(Ranking, how='inner', left_on='home_team', right_on='country_full')
dfinalrank = rankdfhome.merge(Ranking, how='inner', left_on='away_team', right_on='country_full')
dfinalrank.head()

,Group,home_team,away_team,rank_x,country_full_x,country_abrv_x,total_points_x,previous_points_x,rank_change_x,confederation_x,...,rank_y,country_full_y,country_abrv_y,total_points_y,previous_points_y,rank_change_y,confederation_y,rank_date_y,Year_y,Month_y
0,A,Qatar,Ecuador,50,Qatar,QAT,1439.89,1441.97,2,AFC,...,44,Ecuador,ECU,1464.39,1463.74,0,CONMEBOL,2022-10-06,2022,10
1,A,Netherlands,Ecuador,8,Netherlands,NED,1694.51,1679.41,0,UEFA,...,44,Ecuador,ECU,1464.39,1463.74,0,CONMEBOL,2022-10-06,2022,10
2,A,Qatar,Senegal,50,Qatar,QAT,1439.89,1441.97,2,AFC,...,18,Senegal,SEN,1584.38,1584.59,0,CAF,2022-10-06,2022,10
3,A,Ecuador,Senegal,44,Ecuador,ECU,1464.39,1463.74,0,CONMEBOL,...,18,Senegal,SEN,1584.38,1584.59,0,CAF,2022-10-06,2022,10
4,A,Senegal,Netherlands,18,Senegal,SEN,1584.38,1584.59,0,CAF,...,8,Netherlands,NED,1694.51,1679.41,0,UEFA,2022-10-06,2022,10


In [14]:
WorldCup["home_rank"]=dfinalrank["rank_x"]
WorldCup["away_rank"]=dfinalrank["rank_y"]
WorldCup.head()

,Group,home_team,away_team,home_avg_goal,away_avg_goal,home_rank,away_rank,home_age,away_age,home_OVA,...,away_MID,away_DEF,d_Age,d_rank,d_OVA,d_ATT,d_MID,d_DEF,d_avg_goal,Class
0,A,Qatar,Ecuador,NaN,NaN,50,44,26.7,25.8,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,A,Senegal,Netherlands,NaN,NaN,8,44,26.9,25.8,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A,Qatar,Senegal,NaN,NaN,50,18,26.7,25.1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,A,Netherlands,Ecuador,NaN,NaN,44,18,25.8,25.1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,A,Ecuador,Senegal,NaN,NaN,18,8,25.1,26.9,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Point

In [15]:
Point_full=pd.read_csv("/Users/macbook/Desktop/Uni/Quarter 2/ML/World Cup Project/SOFIFA_WorldCup.csv")
Point_full["Name"]=Point_full.Name[:-1]
Point_list=[]
for i in range(len(Point_full)):
    

    if (Point_full.Name.iloc[i] == "Korea Republic" ):
        Point_full.Name.iloc[i] = "South Korea"
        
    if (Point_full.Name.iloc[i] in WC_teams):
        Point_list.append(Point_full.iloc[i])
Point=pd.DataFrame(Point_list)
Point.head()

/var/folders/s5/4b25bvq14pqbm1qxsg0hhlr40000gn/T/ipykernel_50087/16362773.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Point_full.Name.iloc[i] = "South Korea"


,Name,OVA,ATT,MID,DEF,Transfer Budget,Players,Year
0,France,85,86,84,83,€0,36,2023
1,Germany,85,84,85,83,€0,38,2023
2,Brazil,85,85,85,83,€0,40,2023
3,England,84,86,83,83,€0,36,2023
4,Spain,84,84,85,84,€0,40,2023


In [16]:
pointdfhome = dfmatches.merge(Point_full, how='inner', left_on='home_team', right_on='Name')
dfinalpoint = pointdfhome.merge(Point_full, how='inner', left_on='away_team', right_on='Name')
dfinalpoint.head()

,Group,home_team,away_team,Name_x,OVA_x,ATT_x,MID_x,DEF_x,Transfer Budget_x,Players_x,Year_x,Name_y,OVA_y,ATT_y,MID_y,DEF_y,Transfer Budget_y,Players_y,Year_y
0,A,Qatar,Ecuador,Qatar,71,73,71,71,€0,40,2023,Ecuador,75,74,76,75,€0,40,2023
1,A,Netherlands,Ecuador,Netherlands,83,83,83,83,€0,38,2023,Ecuador,75,74,76,75,€0,40,2023
2,A,Qatar,Senegal,Qatar,71,73,71,71,€0,40,2023,Senegal,78,79,77,77,€0,33,2023
3,A,Ecuador,Senegal,Ecuador,75,74,76,75,€0,40,2023,Senegal,78,79,77,77,€0,33,2023
4,A,Senegal,Netherlands,Senegal,78,79,77,77,€0,33,2023,Netherlands,83,83,83,83,€0,38,2023


In [17]:
WorldCup[["home_OVA","home_ATT","home_MID","home_DEF"]]=dfinalpoint[["OVA_x","ATT_x","MID_x","DEF_x"]]
WorldCup[[ "away_OVA","away_ATT","away_MID","away_DEF"]]=dfinalpoint[["OVA_y","ATT_y","MID_y","DEF_y"]]
WorldCup.head()


,Group,home_team,away_team,home_avg_goal,away_avg_goal,home_rank,away_rank,home_age,away_age,home_OVA,...,away_MID,away_DEF,d_Age,d_rank,d_OVA,d_ATT,d_MID,d_DEF,d_avg_goal,Class
0,A,Qatar,Ecuador,NaN,NaN,50,44,26.7,25.8,71,...,76,75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,A,Senegal,Netherlands,NaN,NaN,8,44,26.9,25.8,83,...,76,75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A,Qatar,Senegal,NaN,NaN,50,18,26.7,25.1,71,...,77,77,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,A,Netherlands,Ecuador,NaN,NaN,44,18,25.8,25.1,75,...,77,77,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,A,Ecuador,Senegal,NaN,NaN,18,8,25.1,26.9,78,...,83,83,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Average Goal

In [103]:
WC_teams
Avglist=[]
i=0
for team in WC_teams :
    
    df=pd.read_csv("/Users/macbook/Desktop/Teams/"+str(team)+".csv")
    k=df.d_Goal.iloc[-5:].mean()
    
    Avglist.append([team,k])
    
    i=i+1
    
dfAvg=pd.DataFrame(Avglist, columns= ["Name","Avg_goal"])
#alternative bellow

In [37]:
Avgdfhome = dfmatches.merge(dfAvg, how='inner', left_on='home_team', right_on='Name')
dfinalAvg = Avgdfhome.merge(dfAvg, how='inner', left_on='away_team', right_on='Name')
dfinalAvg.head()

,Group,home_team,away_team,Name_x,Avg_goal_x,Name_y,Avg_goal_y
0,A,Qatar,Ecuador,Qatar,-0.8,Ecuador,0.2
1,A,Netherlands,Ecuador,Netherlands,1.0,Ecuador,0.2
2,A,Qatar,Senegal,Qatar,-0.8,Senegal,0.4
3,A,Ecuador,Senegal,Ecuador,0.2,Senegal,0.4
4,A,Senegal,Netherlands,Senegal,0.4,Netherlands,1.0


In [38]:
WorldCup["home_avg_goal"]=dfinalAvg["Avg_goal_x"]
WorldCup["away_avg_goal"]=dfinalAvg["Avg_goal_y"]
WorldCup.head()

,Group,home_team,away_team,home_avg_goal,away_avg_goal,home_rank,away_rank,home_age,away_age,home_OVA,...,away_MID,away_DEF,d_Age,d_rank,d_OVA,d_ATT,d_MID,d_DEF,d_avg_goal,Class
0,A,Qatar,Ecuador,-0.8,0.2,50,44,26.7,25.8,71,...,76,75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,A,Senegal,Netherlands,1.0,0.2,8,44,26.9,25.8,83,...,76,75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A,Qatar,Senegal,-0.8,0.4,50,18,26.7,25.1,71,...,77,77,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,A,Netherlands,Ecuador,0.2,0.4,44,18,25.8,25.1,75,...,77,77,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,A,Ecuador,Senegal,0.4,1.0,18,8,25.1,26.9,78,...,83,83,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
WorldCup["d_Age"]=WorldCup["home_age"]-WorldCup["away_age"]
WorldCup["d_rank"]=WorldCup["home_rank"]-WorldCup["away_rank"]
WorldCup["d_OVA"]=WorldCup["home_OVA"]-WorldCup["away_OVA"]
WorldCup["d_ATT"]=WorldCup["home_ATT"]-WorldCup["away_ATT"]
WorldCup["d_MID"]=WorldCup["home_MID"]-WorldCup["away_MID"]
WorldCup["d_DEF"]=WorldCup["home_DEF"]-WorldCup["away_DEF"]
WorldCup["d_avg_goal"]=WorldCup["home_avg_goal"]-WorldCup["away_avg_goal"]
WorldCup.head()

,Group,home_team,away_team,home_avg_goal,away_avg_goal,home_rank,away_rank,home_age,away_age,home_OVA,...,away_MID,away_DEF,d_Age,d_rank,d_OVA,d_ATT,d_MID,d_DEF,d_avg_goal,Class
0,A,Qatar,Ecuador,-0.8,0.2,50,44,26.7,25.8,71,...,76,75,0.9,6,-4,-1,-5,-4,-1.0,NaN
1,A,Senegal,Netherlands,1.0,0.2,8,44,26.9,25.8,83,...,76,75,1.1,-36,8,9,7,8,0.8,NaN
2,A,Qatar,Senegal,-0.8,0.4,50,18,26.7,25.1,71,...,77,77,1.6,32,-7,-6,-6,-6,-1.2,NaN
3,A,Netherlands,Ecuador,0.2,0.4,44,18,25.8,25.1,75,...,77,77,0.7,26,-3,-5,-1,-2,-0.2,NaN
4,A,Ecuador,Senegal,0.4,1.0,18,8,25.1,26.9,78,...,83,83,-1.8,10,-5,-4,-6,-6,-0.6,NaN


In [40]:
EvidenceSet=WorldCup[["home_team","away_team","d_Age","d_rank","d_OVA","d_ATT","d_MID","d_DEF","d_avg_goal","Class"]]
#WorldCup.to_csv('/Users/macbook/Desktop/AneeshTeams/FullEvidence.csv', index=False, header=True)
#EvidenceSet.to_csv('/Users/macbook/Desktop/AneeshTeams/EvidenceSet.csv', index=False, header=True)
EvidenceSet.to_csv('/Users/macbook/Desktop/AneeshTeams/EvidenceNewGoal.csv', index=False, header=True)


In [34]:
AVG_GOAL=pd.read_csv("/Users/macbook/Desktop/AvgGoal.csv")
AVG_GOAL.head()

,Name,Avg_goal
0,Ecuador,0.2
1,Netherlands,1.0
2,Qatar,-0.8
3,Senegal,0.4
4,England,0.2


In [35]:
dfAvg=AVG_GOAL

In [36]:
dfAvg.head()

,Name,Avg_goal
0,Ecuador,0.2
1,Netherlands,1.0
2,Qatar,-0.8
3,Senegal,0.4
4,England,0.2
